In [1]:
import pandas as pd

In [162]:
bookings = pd.read_csv('2_bookings.csv', sep=';')

In [17]:
bookings.shape

(119390, 21)

In [166]:
bookings[:7]

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,total_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,0.0
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,0.0
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,0.0
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,0.0
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03,0.0


In [29]:
bookings.dtypes

hotel                         object
is_canceled                    int64
lead_time                      int64
arrival_full_date             object
arrival_date_year              int64
arrival_date_month            object
arrival_date_week_number       int64
arrival_date_day_of_month      int64
stays_in_weekend_nights        int64
stays_in_week_nights           int64
stays_total_nights             int64
adults                         int64
children                     float64
babies                         int64
meal                          object
country                       object
reserved_room_type            object
assigned_room_type            object
customer_type                 object
reservation_status            object
reservation_status_date       object
dtype: object

In [164]:
#приведение имен к нижнему регистру и snale_case
bookings.columns = bookings.columns.str.lower().str.replace(' ', '_')

In [242]:
#топ-5 стран с наибольшим числом бронирований
bookings_by_country_top5 = bookings.query('is_canceled == 0')['country'].value_counts()[:5]

In [59]:
#средняя продолжительность бронирования в зависимости от типа отеля
avg_durings_by_hotel = bookings \
    .groupby('hotel', as_index=False) \
    .aggregate({'stays_total_nights': 'mean'}).round(2)

In [246]:
#количество бронирований с заменой типа комнаты
changings_room_type = bookings \
    .query('assigned_room_type != reserved_room_type') \
    .aggregate({'assigned_room_type': 'count'}) 

In [249]:
#количество бронирований по месяцам в рамках года
bookings_by_years_and_months = bookings \
    .groupby('arrival_date_year')['arrival_date_month'] \
    .value_counts()

In [252]:
#самый популярный месяц бронирования в 2016 году
poplular_month_2016=bookings \
    .query('arrival_date_year == 2016') \
    .groupby('arrival_date_year')['arrival_date_month'] \
    .value_counts() \
    .idxmax()

In [105]:
#самый популярный месяц бронирования в 2017 году
popular_month_2017=bookings \
    .query('arrival_date_year == 2017') \
    .groupby('arrival_date_year')['arrival_date_month'] \
    .value_counts() \
    .idxmax()

In [154]:
#количество отмен бронирований по месяцам в рамках года
bookings_by_years_and_months = bookings \cancelled_bookings_by_month = bookings \
    .query('is_canceled == 1 and hotel == "City Hotel"') \
    .groupby('arrival_date_year')['arrival_date_month'].value_counts()

In [253]:
#среднее количество гостей по возрастной категории на одно бронирование
avg_by_ages = bookings \
    .aggregate({'adults': 'mean','children': 'mean','babies': 'mean'}) \
    .round(2)

In [256]:
#add total_kids
bookings['total_kids'] = bookings.children + bookings.babies 


In [170]:
# наибольшее среднее количество детей на одно бронирование в зависимости от типа отеля
bookings_with_kids_by_types_max = bookings \
    .groupby('hotel') \
    .aggregate({'total_kids': 'mean'}) \
    .max()

In [257]:
#add has_kids
bookings['has_kids'] = bookings.total_kids > 0


In [219]:
#процент отмен бронирования среди гостей с детьми
churn_rate_has_kids = (bookings.query('is_canceled == 1 and has_kids').is_canceled.count()\
     /bookings.query('has_kids').is_canceled.count() \
     * 100) \
    .round(2)

In [222]:
#процент отмен бронирования среди гостей без детей
churn_rate_no_kids = (bookings.query('is_canceled == 1 and has_kids==False').is_canceled.count()\
     /bookings.query('has_kids==False').is_canceled.count() \
     * 100) \
    .round(2)